# Zuora/Eloqua export/import service

## Table of Contents
1. Zuora export service, responsible for exporting any object SQL-style call in CSV/JSON format
More on that here: https://knowledgecenter.zuora.com/DC_Developers/M_Export_ZOQL
2. Zuora integration tests
3. Zuora query, an example query that retrieves the entire list of zuora subscriptions between two dates (start, end)
The resulting JSON is passed directly to the API: https://www.zuora.com/developer/api-reference/#operation/Object_POSTExport
4. eloqua import service, responsible for importing mapped JSON data in to eloqua
5. eloqua create template, this template should match the the output from the eloqua customobject/:id/fields REST call:
More on that here: https://docs.oracle.com/cloud/latest/marketingcs_gs/OMCAC/api-Application-2.0-Custom%20object%20data.html
6. eloqua integration tests
7. mock zuora and eloqua endpoints using express router



zuora export service?



In [ ]:
var importer = require('../Core');
var xlsx = require('xlsx');
var request = importer.import('request polyfill');

function getAuthHeaders(zuoraConfig) {
    return {
        'Content-Type': 'application/json',
        'apiAccessKeyId': zuoraConfig.rest_api_user,
        'apiSecretAccessKey': zuoraConfig.rest_api_password,
        'Accept': 'application/json'
    };
}

function zuoraBulkExport(query, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/object/export',
        json: query,
        method: 'POST',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => r.body.Id)
}

function zuoraBulkExportStatus(exportId, zuoraConfig) {
    console.log('waiting...');
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/object/export/' + exportId,
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => {
        if (r.body.Status === 'Completed') {
            return r.body.FileId;
        } else if (r.body.Status === 'Processing' || r.body.Status === 'Pending') {
            return new Promise(resolve => setTimeout(resolve, 500))
                .then(() => zuoraBulkExportStatus(exportId, zuoraConfig));
        } else {
            throw new Error('Export status error ' + r.statusCode + ' ' + r.body.Status);
        }
    });
}

function zuoraBulkExportFile(fileId, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/files/' + fileId,
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => r.body)
}

function csvToJson(csv) {
    const workbook = xlsx.read(new Buffer(csv), {type:"buffer"});
    return xlsx.utils.sheet_to_json(workbook.Sheets[workbook.SheetNames[0]]);
}

function getCatalog(zuoraConfig, next) {
    var catalog = [];
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + (typeof next !== 'undefined' ? next.replace(/\/v1/ig, '') : '/catalog/products'),
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => {
        catalog = catalog.concat(r.body.products);
        if(r.body.nextPage) {
            return getCatalog(zuoraConfig, r.body.nextPage).then(r => catalog.concat(r));
        }
        return catalog;
    })
}

module.exports = {
    csvToJson,
    zuoraBulkExport,
    zuoraBulkExportFile,
    zuoraBulkExportStatus,
    getCatalog
}


zuora export service test?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var {
    zuoraBulkExport,
    zuoraBulkExportStatus,
    zuoraBulkExportFile,
    csvToJson,
} = importer.import('zuora to eloqua.ipynb[0]');
var request = importer.import('http request polyfill');

var sandbox = sinon.createSandbox();
var zuoraConfig = {
    "rest_api_user":"devteam@fakepage.com",
    "rest_api_password":"pass",
    "rest_api_url": "http://localhost:18888"
};

describe('zuora oauth', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should connect to zuora using oauth', () => {
        const dummyId = '123';
        const dummyQuery = 'SELECT * FROM Accounts';
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {Id: dummyId } }));
        
        return zuoraBulkExport(dummyQuery, zuoraConfig)
            .then(result => {
                assert.equal(result, dummyId);
                assert(requestStub.calledOnce, 'request should only be called once');
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0].json, dummyQuery);
            });
    })
    
    it('should wait for the export to complete', () => {
        const dummyId = '12345'
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {Status: 'Completed', FileId: dummyId } }));
        
        return zuoraBulkExportStatus('123', zuoraConfig)
            .then(result => {
                assert.equal(result, dummyId);
                assert(requestStub.calledOnce, 'request should only be called once');
                //stubCall = requestStub.getCall(0);
                //assert.equal(stubCall.args[0].json, dummyQuery);
            })
    })
    
    it('should download the csv file', () => {
        const csvFile = 'some,csv,file';
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: csvFile }));
        
        return zuoraBulkExportFile('1234', zuoraConfig)
            .then(result => {
                assert.equal(result, csvFile);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should convert CSV to JSON', () => {
        const result = csvToJson('some,csv,file\n1,2,3');
        assert.equal(result[0].some, 1);
    })
    
})



zuora renewals query?


In [ ]:
var moment = require('moment');
var chrono = require('chrono-node');
var excludedRatePlans = [
    'Act! Pro - New License',
    'Act! Pro - 30 Day Support',
    'Act! Pro - Upgrade License',
    'Act! Password Reset Charge',
    'Act! Premium Cloud - Trial',
    'Act! Pro V19 - Upgrade License',
    'Act! Pro V20 - Upgrade License',
]
var excludedProductSkus = [
    '00000006'
]
var currencies = [
    '',
    'USD',
    'AUD',
    'NZD',
]

var query = `SELECT
    Account.Id,
    Account.Name,
    Account.AccountNumber,
    Account.resellerofRecord__c,
    Account.renewalRep__c,
    Account.commisionedSalesRep__c,
    Account.CreatedDate,
    Account.Currency,
    SoldToContact.WorkEmail,
    SoldToContact.Country,
    SoldToContact.State,
    BillToContact.WorkEmail,
    RatePlan.Id,
    RatePlan.Name,
    RatePlanCharge.Id,
    RatePlanCharge.BillingPeriod,
    RatePlanCharge.Description,
    RatePlanCharge.Quantity,
    RatePlanCharge.Version,
    RatePlanCharge.CreatedDate,
    RatePlanCharge.EffectiveEndDate,
    DefaultPaymentMethod.CreditCardExpirationMonth,
    DefaultPaymentMethod.CreditCardExpirationYear,
    DefaultPaymentMethod.CreditCardMaskNumber,
    ProductRatePlanCharge.Id,
    ProductRatePlan.planType__c,
    ProductRatePlan.planSubType__c,
    ProductRatePlan.Id,
    ProductRatePlan.Name,
    Product.productType__c,
    Product.Name,
    Product.Description,
    Product.Id,
    Product.SKU,
    Subscription.Id,
    Subscription.Name,
    Subscription.Status,
    Subscription.Reseller__c,
    Subscription.SubscriptionEndDate,
    Subscription.SubscriptionStartDate,
    Subscription.TermStartDate,
    Subscription.TermEndDate,
    Subscription.AutoRenew
FROM RatePlanCharge
WHERE Subscription.Status!='Draft' AND Subscription.Status!='Cancelled' AND Subscription.Status!='Expired'
    AND Subscription.TermEndDate &gt;='{0}' AND Subscription.TermEndDate &lt;='{1}'
    AND (Account.Currency='${currencies.join("' OR Account.Currency='")}')
    AND (ProductRatePlan.Name!='${excludedRatePlans.join("' AND ProductRatePlan.Name!='")}')
    AND (Product.SKU!='${excludedProductSkus.join("' AND Product.SKU!='")}')
    AND NOT (SoldToContact.WorkEmail LIKE 'qaaw%@gmail.com')
    AND NOT (BillToContact.WorkEmail LIKE 'qaaw%@gmail.com')
    AND NOT (Account.Name LIKE '%do not use%')
`;
// AND (RatePlanCharge.EffectiveEndDate &gt;='{2}' OR RatePlanCharge.ChargeType='OneTime')
// removed this so that discounts show up on the account
// AND RatePlanCharge.BillingPeriod!='Month'

function getQuery(start, end) {
    // TODO: add updated option for pulling based on subscription term or based on modified fields
    // TODO: add a query here to just look up one subscription record by it's ID?
    return {
        Query: query.replace('{0}', moment(chrono.parseDate(start)).format('YYYY-MM-DD'))
                    .replace('{1}', moment(chrono.parseDate(end)).format('YYYY-MM-DD'))
                    .replace('{2}', moment(new Date()).format('YYYY-MM-DD')),
        Format: 'csv',
        Zip: false
    };
}
module.exports = getQuery;


test zuora renewals query?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getQuery = importer.import('zuora renewals query');

describe('zuora query', () => {
    it('should include the dates specified', () => {
        const now = new Date();
        const year = now.getMonth() < 11 ? (now.getFullYear() - 1) : now.getFullYear()
        const q = getQuery('beginning of November', 'beginning of December');
        assert(q.Query.includes(year + '-11-01'), 'should have correct dates');
    })
})


eloqua import service?


In [ ]:
var importer = require('../Core');
var request = importer.import('http request polyfill');
var {
    createTemplate
} = importer.import('eloqua create template');

function eloquaOauth(eloquaConfig) {
    if (typeof eloquaConfig === 'undefined'
        || eloquaConfig === null
        || typeof eloquaConfig.rest_api_company === 'undefined'
        || typeof eloquaConfig.rest_api_user === 'undefined'
        || typeof eloquaConfig.rest_api_password === 'undefined'
        || typeof eloquaConfig.rest_client_id === 'undefined'
        || typeof eloquaConfig.rest_secret === 'undefined') {
        throw new Error('Please supply valid config eloqua configuration.');
    }
    var authBody = {
        "grant_type": "password",
        "scope": "full",
        "username": eloquaConfig.rest_api_company + '\\' + eloquaConfig.rest_api_user,
        "password": eloquaConfig.rest_api_password
    };
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.token_uri,
        method: 'POST',
        json: authBody,
        headers: {
            'Authorization': "Basic " + new Buffer(eloquaConfig.rest_client_id + ":" + eloquaConfig.rest_secret).toString("base64"),
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    }).then(res => {
        res.body.expires = (new Date()).getTime() + parseFloat(res.body.expires_in) * 1000;
        return res.body;
    });
}

function eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig) {
    console.log(syncUri);
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + syncUri,
        method: 'GET',
        headers: {
            'Authorization': "Bearer " + eloquaToken.access_token,
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    }).then(r => {
        if (r.body.status === 'success' || r.body.status === 'warning') {
            return true;
        } else if (r.body.status === 'active' || r.body.status === 'pending') {
            return new Promise(resolve => setTimeout(resolve, 500))
                .then(() => eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig));
        } else {
            throw new Error('Sync status error ' + r.statusCode + ' ' + JSON.stringify(r.body));
        }
    });
}

function eloquaBulkImportSync(importUri, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/syncs',
        method: 'POST',
        json: {
            syncedInstanceUri: importUri
        },
        headers: {
            'Authorization': "Bearer " + eloquaToken.access_token,
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    }).then(r => {
        const syncUri = r.body.uri;
        return eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig);
    });
}

function eloquaBulkImportData(json, importUri, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + importUri + '/data',
        method: 'POST',
        json: json,
        headers: {
            'Authorization': "Bearer " + eloquaToken.access_token,
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    });
}

// TODO: update to custom data object
// https://docs.oracle.com/cloud/latest/marketingcs_gs/OMCAB/Developers/BulkAPI/Endpoints/Custom%20objects/Imports/post-customObjects-imports.htm
function eloquaBulkImport(eloquaToken, eloquaConfig) {
    // TODO: move this to separate function/callout
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/customobjects/60/imports',
        method: 'POST',
        json: createTemplate,
        headers: {
            'Authorization': "Bearer " + eloquaToken.access_token,
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    }).then(res => {
        return res.body.uri;
    });
}


module.exports = {
    eloquaBulkImport,
    eloquaBulkImportData,
    eloquaBulkImportSync,
    eloquaBulkImportStatus,
    eloquaOauth
}


test eloqua import service?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var {
    eloquaOauth,
    eloquaBulkImport,
    eloquaBulkImportData,
    eloquaBulkImportSync
} = importer.import('eloqua import service');

var {
    getEloquaConfig,
    getOauthToken,
    getImportData
} = importer.import('eloqua import blueprints');
var request = importer.import('http request polyfill');

var eloquaConfig = getEloquaConfig();
var eloquaToken = getOauthToken();
var sandbox = sinon.createSandbox();

describe('eloqua bulk upload', () => {
        
    afterEach(() => {
        sandbox.restore();
    })

    it('should get a valid oauth token', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {expires_in: 1000 } }));
        
        return eloquaOauth(eloquaConfig)
            .then(r => {
                assert(r.expires > (new Date()).getTime());
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should create a bulk import instance', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123' } }));
        
        return eloquaBulkImport(eloquaToken, eloquaConfig)
            .then(r => {
                assert(r.includes('/imports/'));
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should update data to eloqua', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123' }, statusCode: 204 }));
        
        return eloquaBulkImportData([getImportData()], '/imports/123', eloquaToken, eloquaConfig)
            .then(r => {
                assert(r.statusCode === 204, 'invalid status recieved from import ' + r.statusCode);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should verify upload was successful', () => {
        var importUri;
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123', status: 'success' } }));
        
        return eloquaBulkImportSync(importUri, eloquaToken, eloquaConfig)
            .then(r => {
                assert(r === true);
                assert(requestStub.called, 'request should only be called once');
            })
    })
    
})


zuora eloqua mapper?



In [ ]:
var moment = require('moment');
var _ = require('lodash');
var chrono = require('chrono-node');

function mapRatePlanToProduct(description) {
    if(description.includes('trial') > -1)
        return 'trial';
    else if (description.includes('volume') || description.includes('discount'))
        return 'discount';
    else if (description.includes('cloud') && !description.includes('trial'))
        return 'actpremiumcloud';
    else if (description.includes('premium') && !description.includes('cloud')
             && !description.includes('trial') && !description.includes('support'))
        return 'actpremium';
    else if (description.includes('pro')  && !description.includes('support'))
        return 'actpro';
    else if (description.includes('support'))
        return 'support';
    else if (description.includes('handheld'))
        return 'handheld';
    else if (description.includes('aem'))
        return 'aem';
}

function mapDataToFields(records) {
    var uniqueIds = _.groupBy(records, r => r['Account.Id']);
    return Object.keys(uniqueIds).map(k => {
        const rpcs = _.groupBy(uniqueIds[k], r => r['RatePlanCharge.Id']);
        const charges = Object.keys(rpcs).map(k => _.sortBy(rpcs[k], r => r['RatePlanCharge.Version']).pop());
        const record = {};
        charges.sort((a, b) =>
                          chrono.parseDate(b['Subscription.TermEndDate']).getTime()
                          - chrono.parseDate(a['Subscription.TermEndDate']).getTime());
        // contact information
        const contact = charges.filter(p => p['SoldToContact.WorkEmail'] || p['BillToContact.WorkEmail'])[0]
        if(typeof contact === 'undefined') {
            console.log(charges);
            return;
        }
        record['EmailAddress'] = contact['SoldToContact.WorkEmail'] || contact['BillToContact.WorkEmail'];
        record['State'] = contact['SoldToContact.State'];
        record['Country'] = contact['SoldToContact.Country'];
        record['Currency'] = contact['Account.Currency'];

        // primary product on subscription
        const actProduct = charges.filter(p => {
            const pname = mapRatePlanToProduct(p['ProductRatePlan.Name'].toLowerCase());
            return pname === 'actpremiumcloud' || pname === 'actpremium' || pname === 'actpro' || pname === 'trial'
        })[0];
        
        if(typeof actProduct !== 'undefined') {
            record['ActProduct'] = mapRatePlanToProduct(actProduct['ProductRatePlan.Name'].toLowerCase());
            record['Quantity'] = actProduct['RatePlanCharge.Quantity'];
        } else {
            record['ActProduct'] = 'Unknown';
            record['Quantity'] = 0;
        }
        
        // discounts!
        const discount = charges.filter(p => {
            const pname = mapRatePlanToProduct(p['ProductRatePlan.Name'].toLowerCase());
            return pname === 'discount'
        })[0];
        if(typeof discount !== 'undefined') {
            record['Discount'] = actProduct['ProductRatePlan.Name'];
        } else {
            record['Discount'] = '';
        }
        
        // support
        const support = charges.filter(p => {
            return mapRatePlanToProduct(p['ProductRatePlan.Name'].toLowerCase()) === 'support'
        })[0];
        if(typeof support !== 'undefined') {
            record['Support'] = mapRatePlanToProduct(support['ProductRatePlan.Name'].toLowerCase());
            record['SupportQuantity'] = support['RatePlanCharge.Quantity'];
        } else {
            record['Support'] = 'Unknown';
            record['SupportQuantity'] = 0;
        }
        
        // subscription data
        const renewal = chrono.parseDate(charges[0]['Subscription.TermEndDate']);
        record['RenewalsStatus'] = charges[0]['Subscription.Status'];
        record['RenewalDate'] = moment(renewal).format('YYYY-MM-DD');
        
        // TODO: add card expiration
        const expiration = new Date();
        expiration.setDate(1);
        expiration.setMonth(parseInt(charges[0]['DefaultPaymentMethod.CreditCardExpirationMonth']) - 1);
        expiration.setYear(parseInt(charges[0]['DefaultPaymentMethod.CreditCardExpirationYear']));
        record['CardExpiration'] = moment(expiration).format('YYYY-MM-DD');
        record['Last4DigitsOfCard'] = ((/([0-9]+)/ig).exec(charges[0]['DefaultPaymentMethod.CreditCardMaskNumber']) || [])[1];
        
        // account data
        record['RepName'] = charges[0]['Account.renewalRep__c'];
        record['RORName'] = charges[0]['Account.resellerofRecord__c'];
        record['RORNumber'] = ((/([0-9]+)/ig).exec(charges[0]['Account.resellerofRecord__c']) || [])[1];
        record['AccountId'] = charges[0]['Account.Id'];

        return record;
    }).filter(r => typeof r !== 'undefined');
}
module.exports = mapDataToFields;


zuora eloqua mapper test?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var sandbox = sinon.createSandbox();

describe('map zuora data fields', () => {
    it('should map basic data', () => {
        
    })
    
    it('should map contact data', () => {
        
    })
    
    it('should map support data', () => {
        
    })
    
    it('should map cancelled data', () => {
        
    })
})


eloqua import create template?


In [ ]:

// TODO: replace ID for CDO, shouldn't always be 60
module.exports = {
    bulkImportTemplate,
    contentCreateTemplate,
    temporaryImportTemplate
}

function bulkImportTemplate() {
    return {
        "name": "Renewals Micro-service - Bulk Import",
        "mapDataCards": "true",
        "mapDataCardsEntityField": "{{Contact.Field(C_EmailAddress)}}",
        "mapDataCardsSourceField": "EmailAddress",
        "mapDataCardsEntityType": "Contact",
        "mapDataCardsCaseSensitiveMatch": "false",
        "updateRule": "always",
        "fields": {
            "AccountId": "{{CustomObject[60].Field(Account_ID1)}}",
            "ActProduct": "{{CustomObject[60].Field(Act_Product1)}}",
            "EmailAddress": "{{CustomObject[60].Field(Email_Address1)}}",
            "Quantity": "{{CustomObject[60].Field(Quantity1)}}",
            "Support": "{{CustomObject[60].Field(Support1)}}",
            "SupportQuantity": "{{CustomObject[60].Field(Support_Quantity1)}}",
            "RenewalDate": "{{CustomObject[60].Field(Renewal_Date1)}}",
            "RenewalsStatus": "{{CustomObject[60].Field(Renewal_Status1)}}",
            "RepName": "{{CustomObject[60].Field(Rep_Name1)}}",
            "RORName": "{{CustomObject[60].Field(ROR_Name1)}}",
            "RORNumber": "{{CustomObject[60].Field(ROR_Number1)}}",
            "CardExpiration": "{{CustomObject[60].Field(Card_Expiration1)}}",
            "State": "{{CustomObject[60].Field(State1)}}",
            "Country": "{{CustomObject[60].Field(Country1)}}",
            "Currency": "{{CustomObject[60].Field(Currency1)}}"
        },
        "identifierFieldName": "EmailAddress"
    }
}

function contentCreateTemplate() {
    return {
        "recordDefinition": {
            "ContactID": "{{Contact.Id}}",
            "EmailAddress": "{{Contact.Field(C_EmailAddress)}}"
        },
        "height": 256,
        "width": 256,
        "editorImageUrl": "https://purchasesprint.actops.com/assets/act-logo-circle.png",
        "requiresConfiguration": false,
    }
}

function temporaryImportTemplate(instance, execution) {
    return {
        "name": "Renewals Micro-service - Bulk Import",
        "mapDataCards": "true",
        "mapDataCardsEntityField": "{{Contact.Field(C_EmailAddress)}}",
        "mapDataCardsSourceField": "EmailAddress",
        "mapDataCardsEntityType": "Contact",
        "mapDataCardsCaseSensitiveMatch": "false",
        "updateRule": "always",
        "fields": {
            "EmailAddress": "{{Contact.Field(C_EmailAddress)}}",
            "Last4DigitsOfCard": "{{Contact.Field(Last_4_Digits_of_Card1)}}",
            "Content": `{{ContentInstance(${instance}).Execution[${execution}]}}`
        },
        "identifierFieldName": "EmailAddress"
    }
}


test eloqua import create template?


In [ ]:
describe('eloqua bulk import definition', () => {
    it('should return the current import definition', () => {
        
    })
})


eloqua import blueprints?



In [ ]:

function getImportData() {
    return {
        "AccountId": '1234',
        "ActProduct": 'premium',
        "EmailAddress": "brian.cullinan@swiftpage.com",
        "ExpirationMonth": '12',
        "ExpirationYear": '2017',
        "Last4DigitsOfCard": '1234',
        "Quantity": 5,
        "RenewalDate": '11/12/2017',
        "RenewalsStatus": "Drew loves marketing!",
        "RepName": 'Brian',
        "RORName": 'James',
        "RORNumber": '1111',
        "State": 'AZ',
    }
}

function getOauthToken() {
    return {
        "access_token": "access_token",
        "token_type": "bearer",
        "expires_in": 28800,
        "refresh_token": "refresh_token",
        "expires": 1515206930614
    }
}

function getEloquaConfig() {
    return {
        "authorize_uri":"http://localhost:18888/auth/oauth2/authorize",
        "token_uri":"http://localhost:18888/auth/oauth2/token",
        "rest_api_url":"http://localhost:18888",
        "rest_client_id":"client-id",
        "rest_secret":"secret",
        "rest_api_company": "swiftpage",
        "rest_api_user":"username",
        "rest_api_password":"password"
    }
}
module.exports = {
    getImportData,
    getOauthToken,
    getEloquaConfig
};


use the eloqua rest api to retrieve a list of objects


In [ ]:
var assert = require('assert');
var util = require('util');
var importer = require('../Core');
var request = importer.import('http request polyfill');
var {
    zuoraBulkExport,
    zuoraBulkExportStatus,
    zuoraBulkExportFile,
    csvToJson
} = importer.import('zuora to eloqua.ipynb[0]');
var {eloquaOauth} = importer.import('eloqua import service');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';

var eloquaToken, eloquaConfig;
function eloquaCustomObjects() {
    eloquaConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_production.json').toString().trim());
    return Promise.resolve(typeof eloquaToken !== 'undefined'
                    && eloquaToken.expires > (new Date()).getTime()
                    ? eloquaToken
                    : eloquaOauth(eloquaConfig))
        .then(r => {
            eloquaToken = r;
            fs.writeFileSync(
                    PROFILE_PATH + '/.credentials/eloqua_token.json',
                    JSON.stringify(r, null, 4));
            assert(r.expires > (new Date()).getTime());
            return request({
                followAllRedirects: true,
                uri: eloquaConfig.rest_api_url + '/bulk/2.0/customobjects/60/imports', // /60/fields',
                method: 'GET',
                headers: {
                    'Authorization': "Bearer " + eloquaToken.access_token,
                    'Content-Type': 'application/json',
                    'Accept': 'application/json'
                }
            })
        })
}
module.exports = eloquaCustomObjects;

if(typeof $$ !== 'undefined') {
    $$.async();
    eloquaCustomObjects()
    /*
        .then(r => {
            // delete import definitions
            const imports = JSON.parse(r.body).items;
            return importer.runAllPromises(imports.map(r => resolve => {
                return request({
                    followAllRedirects: true,
                    uri: eloquaConfig.rest_api_url + '/bulk/2.0' + r.uri,
                    method: 'DELETE',
                    headers: {
                        'Authorization': "Bearer " + eloquaToken.access_token,
                        'Content-Type': 'application/json',
                        'Accept': 'application/json'
                    }
                }).then(r => resolve(r)).catch(e => resolve(e))
            }))
        })
        */
        .then(r => $$.mime({'text/html': '<pre>' + JSON.stringify(JSON.parse(r.body), null, 4) + '</pre>'}))
        .catch(e => $$.sendError(e))
}


// TODO: find other definitions, compare, and import using the same definition



aws entry point?

In [ ]:
var importer = require('../Core');
var bulkUploadEloqua = importer.import('bulk upload eloqua');
var getZuoraMonth = importer.import('zuora export month');
var mapDataToFields = importer.import('zuora eloqua mapper');

function handler(event, context, callback) {
    const zuoraConfig = {
        "rest_api_user": process.env.ZUORA_API_USER,
        "rest_api_password": process.env.ZUORA_API_PASS,
        "rest_api_url": process.env.ZUORA_API_URL
    };
    return module.exports.getZuoraMonth(0, zuoraConfig)
        .then(records => module.exports.mapDataToFields(records))
        .then(accounts => module.exports.bulkUploadEloqua(accounts))
        .then(() => callback(null, {
            'statusCode': 200,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Success!'
        }))
        .catch(e => callback(e, {
            'statusCode': 500,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Error: ' + e.message
        }))
}
module.exports = {
    handler,
    getZuoraMonth,
    mapDataToFields,
    bulkUploadEloqua
};


test aws entry point?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var bundle = importer.import('aws entry point');

process.env.ZUORA_API_USER = 'devteam@fakepage.com';
process.env.ZUORA_API_USER = 'pass';
process.env.ZUORA_API_USER = 'http://localhost:18888';
var sandbox = sinon.createSandbox();

describe('bulk upload entry point', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora month export', () => {
        var callback = sinon.spy();
        sandbox.stub(bundle, 'getZuoraMonth').returns(Promise.resolve([]));
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(bundle.getZuoraMonth.calledWithMatch(0));
            });
    });
    
    it('should call data mapper', () => {
        var callback = sinon.spy();
        sandbox.stub(bundle, 'getZuoraMonth').returns(Promise.resolve([]));
        sandbox.stub(bundle, 'mapDataToFields').returns([]);
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(bundle.mapDataToFields.calledWithMatch([]));
            });
    });
    
    it('should call bulk upload', () => {
        var callback = sinon.spy();
        sandbox.stub(bundle, 'getZuoraMonth').returns(Promise.resolve([]));
        sandbox.stub(bundle, 'mapDataToFields').returns([]);
        sandbox.stub(bundle, 'bulkUploadEloqua').returns([]);
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(bundle.bulkUploadEloqua.calledWithMatch([]));
            });
    });
})


notify entry point?


In [ ]:
var importer = require('../Core');
var getZuoraAccounts = importer.import('zuora account service');
var bulkUploadEloqua = importer.import('bulk upload eloqua');

function handler(event, context, callback) {
    var body = event || {};
    try {
        if (event.body || event.queryStringParameters) {
            body = Object.assign(event.body || {}, event.queryStringParameters || {});
        }
    } catch(e) {
        callback(e, {
            'statusCode': 500,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify({'Error': e.message})
        })
        return;
    }
    // TODO: parse action and call from notify service or call with posted data?
    // TODO: add an entry point for Zuora subscription callout to update single records in eloqua?
    return module.exports.getZuoraAccounts(body)
        .then(accounts => module.exports.bulkUploadEloqua(accounts, body.instanceId, body.executionId))
        .then(() => callback(null, {
            'statusCode': 200,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Success!'
        }))
        .catch(e => callback(e, {
            'statusCode': 500,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Error: ' + e.message
        }))
}
module.exports = {
    handler,
    getZuoraAccounts,
    bulkUploadEloqua
};


test notify entry point?


In [ ]:
var sinon = require('sinon');
var sandbox = sinon.createSandbox();
var importer = require('../Core');
var bundle = importer.import('notify entry point');
var assert = require('assert');

describe('content notify entry point', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora export', () => {
        const callback = sinon.spy();
        const dummyBody = {
        }
        
        const requestStub = sandbox.stub(bundle, "getZuoraAccounts")
            .returns(Promise.resolve([]));
        sandbox.stub(bundle, "bulkUploadEloqua")
            .returns(Promise.resolve(true));
        
        return bundle.handler(dummyBody, null, callback)
            .then(() => {
                assert(callback.calledOnce);
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], dummyBody);
            });
    })
    
    it('should call bulk upload', () => {
        const callback = sinon.spy();
        const dummyAccounts = [];
        
        sandbox.stub(bundle, "getZuoraAccounts")
            .returns(Promise.resolve(dummyAccounts));
        const requestStub = sandbox.stub(bundle, "bulkUploadEloqua")
            .returns(Promise.resolve(true));
        
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(callback.calledOnce);
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], dummyAccounts);
            });
    })
})


zuora export month?



In [1]:
var importer = require('../Core');
var getQuery = importer.import('zuora renewals query');
var {
    zuoraBulkExport,
    zuoraBulkExportStatus,
    zuoraBulkExportFile,
    csvToJson,
} = importer.import('zuora to eloqua.ipynb[0]');

function getZuoraMonth(months, zuoraConfig) {
    if(!months) {
        months = 0;
    }
    var start = new Date();
    start.setMonth(start.getMonth() - months);
    start.setDate(1);
    start.setHours(0, 0, 0);
    var end = new Date();
    end.setMonth(end.getMonth() + months + 1);
    end.setDate(1);
    end.setHours(0, 0, 0);
    if(start.getMonth() > end.getMonth()) {
        end.getFullYear(end.getFullYear() + 1)
    }
    
    console.log(getQuery(start.toString(), end.toString()));
    
    return zuoraBulkExport(getQuery(start.toString(), end.toString()), zuoraConfig)
        .then(exportId => zuoraBulkExportStatus(exportId, zuoraConfig))
        .then(fileId => zuoraBulkExportFile(fileId, zuoraConfig))
        .then(r => csvToJson(r))
}
module.exports = {
    getZuoraMonth,
    zuoraBulkExport,
    zuoraBulkExportStatus,
    zuoraBulkExportFile,
    csvToJson
};


SyntaxError: Unexpected token ;

test zuora export month?


In [ ]:
var sinon = require('sinon');
var sandbox = sinon.createSandbox();

describe('zuora export month', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should get the query', () => {
        
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {Id: dummyId } }));

        
    })
    
    it('should call bulk export service', () => {
        
    })
    
    it('should convert csv dump to json', () => {
        
    })
})


zuora account service?


In [ ]:
function getZuoraAccounts(notifyRequest, zuoraConfig) {
    
}
module.exports = getZuoraAccounts;


test zuora account service?


In [ ]:


describe('zuora account service', () => {
    it('should call zuora export', () => {
        
    })
})


bulk upload eloqua?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {
    eloquaOauth,
    eloquaBulkImport,
    eloquaBulkImportData,
    eloquaBulkImportSync
} = importer.import('bulk eloqua import');

function bulkUploadEloqua(accounts, eloquaConfig) {
    var eloquaToken, importUri;
    return eloquaOauth(eloquaConfig)
        .then(r => {
            eloquaToken = r;
            assert(r.expires > (new Date()).getTime());
            return eloquaBulkImport(eloquaToken, eloquaConfig)
        })
        .then(r => {
            importUri = r;
            return eloquaBulkImportData(accounts, importUri, eloquaToken, eloquaConfig);
        })
        .then(() => eloquaBulkImportSync(importUri, eloquaToken, eloquaConfig))
        .then(() => accounts)
        .catch((e) => console.log(e))
}
module.exports = bulkUploadEloqua;

if(typeof $$ !== 'undefined') {
    $$.async();
    bulkUploadEloqua(6)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


test bulk upload eloqua?


In [ ]:
describe('eloqua bulk upload', () => {
    it('should call oauth', () => {
        
    })
    
    it('should call import data', () => {
        
    })
})


sync zuora eloqua end to end?



In [ ]:

describe('zuora to eloqua', () => {
    beforeEach(() => {
    })
    
    it('should export a month of zuora data', () => {
    })
    
    it('should match all products in zuora catalog', () => {
    })
    
    it('should transfer data end-to-end', () => {
    })
    
    /*
    TODO: 
Start a trial through portal
Add / remove support
ACC / distributor updates
Make a purchase
Purchase expires
Change quantity
Cancel subscription
Change quantity within 30 days of renewal
Change quantity within 60 days of renewal
AU/NZ renewals
ACC have eternally free terms
Renew with new subscription, same account
Renew new subscription, same email
*/
    
})


calculate price?


In [ ]:
// given a list of subscription IDs and products, calculate the new subscription total using a catalog export and compare with the preview API

// returns [account number, subtotal, previous discount]
function calculatePrice(subscription, products) {
    const rpcs = _.groupBy(subscription, r => r['RatePlanCharge.Id']);
    const charges = Object.keys(rpcs).map(k => _.sortBy(rpcs[k], r => r['RatePlanCharge.Version']).pop());
    var subtotal = 0;
    var discount;
    var quantity = 0;
    charges
        // TODO: escelate this to mapDataToFields function?
        .filter(c => !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('perpetual'))
        .forEach(charge => {
            const product = products.filter(p => p['id'] === charge['Product.Id'])[0];
            const productRatePlan = product.productRatePlans.filter(p => p['id'] === charge['ProductRatePlan.Id'])[0];
            // select correct price plan for the item
            const productCharge = productRatePlan.productRatePlanCharges.filter(p => p['id'] === charge['ProductRatePlanCharge.Id'])[0];
            const pricing = productCharge.pricing.filter(p => p['currency'] === charge['Account.Currency'])[0];
        
        
            if((charge['ProductRatePlan.Name'] || '').toLowerCase().includes('discount')
                                    || (charge['ProductRatePlan.Name'] || '').toLowerCase().includes('volume')) {
                // TODO: add handler for quantity based discounts?
                discount = pricing.discountPercentage / 100;
            } else {
                const price = pricing.tiers === null
                    ? pricing.price
                    : pricing.tiers.filter(t => charge['RatePlanCharge.Quantity'] >= t['startingUnit']
                                           && charge['RatePlanCharge.Quantity'] <= t['endingUnit'])[0];
                if(typeof price === 'undefined') {
                    throw new Error('unknown rate plan component');
                }
                quantity += parseInt(charge['RatePlanCharge.Quantity']);
                subtotal += parseInt(charge['RatePlanCharge.Quantity']) * price;
            }
        });
    const discounts = charges.filter(c => (c['ProductRatePlan.Name'] || '').toLowerCase().includes('discount') > -1
                                    || (c['ProductRatePlan.Name'] || '').toLowerCase().includes('volume') > -1)
                             .filter(c => !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('diamond')
                                    && !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('distribution'));
    assert(!isNaN(subtotal), 'not a number! ' + JSON.stringify(charges, null, 4))
    //assert(discounts.length <= 1, 'multiple discounts! ' + JSON.stringify(discounts, null, 4))
    return [subscription[0]['Account.AccountNumber'], subtotal, discount, quantity];
}
module.exports = calculatePrice;


calculate price test?


In [5]:
var _ = require('lodash');
var assert = require('assert');
var xlsx = require('xlsx');
var importer = require('../Core');
var fs = require('fs');

var getZuoraMonth = importer.import('zuora export month');
var calculatePrice = importer.import('calculate price');
var {getCatalog} = importer.import('zuora to eloqua.ipynb[0]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString().trim());

function filterROR(accountROR) {
    return [
            '4001372618',
            '4000919381',
            '411183297',
            '4001358862',
            '4000919006',
            '4000919116',
            '4001372618',
            '411182712',
            '411183101',
            '4001635120',
            '4000919342',
            '4000919068'].filter(ror => (accountROR || '').includes(ror)).length > 0
}

function rorsToAccounts(records) {
    return records.filter(a => a['Account.resellerofRecord__c'])
        .filter(a => filterROR(a['Account.resellerofRecord__c']))
        .map(a => a['Account.AccountNumber']);
}

function totalFilteredRecords(zuoraRecords) {
    const rors = rorsToAccounts(zuoraRecords);
    console.log(rors.length + ' ROR accounts with renewing subscriptions out of ' + zuoraRecords.length + ' records in the dump');
    const recordsToValidate = zuoraRecords
        .filter(s => {
            // TODO: graduate these filters to the main export / import?
            return new Date(s['Subscription.TermEndDate']).getTime() < (new Date('4/1/2018')).getTime()
                && new Date(s['Subscription.TermEndDate']).getTime() >= (new Date('2/27/2018')).getTime()
                && s['Account.Currency'] === 'USD'
                && s['RatePlanCharge.BillingPeriod'] === 'Annual' 
                && !s['ProductRatePlan.Name'].toLowerCase().includes('pro')
        })
    const recordsWithRors = recordsToValidate
        .filter(s => rors.includes(s['Account.AccountNumber']));
    console.log(recordsToValidate.length + ' zuora records to validate');
    console.log(recordsWithRors.length + ' of those are RORs and will also be excluded');
    console.log((recordsToValidate.length - recordsWithRors.length) + ' total');
    return recordsToValidate
        .filter(s => !rors.includes(s['Account.AccountNumber']))
}

function accountTotals(zuoraRecords) {
    const uniqueIds = _.groupBy(zuoraRecords, r => r['Account.Id']);
    console.log(Object.keys(uniqueIds).length + ' unique accounts with expiring subscriptions in February');
    const totals = Object.keys(uniqueIds)
        .map(accountId => {
            return calculatePrice(uniqueIds[accountId], catalog);
        });
    return totals;
}

function verifyMissing(worksheet, totals) {
    
    // TODO: compare with zuora preview API and Jacob's spreadsheet
    const accountIds = totals.map(t => t[0]);
    const worksheetIds = worksheet.map(t => t['Account Number']);
    const missing = worksheet.filter(a => !accountIds.includes(a['Account Number']));
    console.log(missing.length + ' accounts in worksheet, not in zuora export: ');
    //console.log(missing.slice(0, 20));
    const missingZuora = accountIds.filter(a => !worksheetIds.includes(a));
    console.log(missingZuora.length + ' accounts in zuora, missing from worksheet: ');
    console.log(missingZuora.slice(0, 20));
    const verifiableWorksheet = worksheet.filter(a => !missing.includes(a));
    const verifiableTotals = totals.filter(t => worksheetIds.includes(t[0]));
    console.log(verifiableWorksheet.length + ' = ' + verifiableTotals.length + ' verifiable records')
    return {verifiableWorksheet, verifiableTotals};
}

function validateWorksheet(calculatedTotals, zuoraRecords) {
    var workbook = xlsx.readFile(PROFILE_PATH + '/Documents/Marketing_File_Mar_.xlsx');
    var worksheet = xlsx.utils.sheet_to_json(workbook.Sheets['Marketing_File_Mar_']);
    console.log(worksheet.length + ' rows in worksheet');
    const worksheetUSD = worksheet.filter(t => t['Currency'] === 'USD');
    console.log(worksheetUSD.length + ' rows are USD');
    const worksheetFiltered = worksheetUSD.filter(t => filterROR(t['ROR Number'] || '') === false);
    console.log(worksheetFiltered.length + ' rows are USD and not ROR');
    
    const {verifiableWorksheet, verifiableTotals} = verifyMissing(worksheetFiltered, calculatedTotals);
    
    const correct = verifiableWorksheet.filter(a => {
        const realTotal = parseFloat((a['Total 1:'] || '').replace(/[\$,\s]/ig, ''));
        const newTotal = parseFloat((a['Total'] || '').replace(/[\$,\s]/ig, ''));
        const oldTotal = verifiableTotals.filter(t => t[0] === a['Account Number'])[0] || [];
        // TODO: decide what to do with discounts?
        if(a['Account Number'] === 'A00191395') {
            console.log(newTotal);
            console.log(oldTotal);
            console.log(oldTotal[1] - (oldTotal[2] ? (oldTotal[1] * oldTotal[2]) : 0))
        }
        return realTotal === oldTotal[1]
            || newTotal === oldTotal[1]
            || newTotal === oldTotal[1] - (oldTotal[2] ? (oldTotal[1] * oldTotal[2]) : 0)
            || newTotal === oldTotal[1] - (oldTotal[1] * .05);
    });
    const incorrect = verifiableWorksheet.filter(a => !correct.includes(a));
    console.log(incorrect.length + ' incorrect, correct: ' + correct.length + ' out of ' + verifiableTotals.length
                + ' - ' + Math.round(correct.length / verifiableTotals.length * 100) + '%');
    // TODO: fix this
    console.log(incorrect.length + ' + ' + correct.length + ' = ' + (incorrect.length + correct.length));
    // TODO: calculate perpetual price for previousinvoice price comparison?
    // TODO: how to handle australia?
    // TODO: how to handle not in worksheet?
    return {correct, incorrect};
}

var catalog, records;
function compareRecordsCatalog() {
    return (typeof catalog !== 'undefined' ? Promise.resolve(catalog) : getCatalog(zuoraConfig))
        .then(r => (catalog = r))
        .then(() => typeof records !== 'undefined' ? Promise.resolve(records) : getZuoraMonth(6, zuoraConfig))
        .then(r => {
            // filter out the records we aren't validating
            records = r;
            const recordsToValidate = totalFilteredRecords(records);
            const zuoraTotals = accountTotals(recordsToValidate);
            
            const {correct, incorrect} = validateWorksheet(zuoraTotals);
        
            const displayIncorrect = incorrect.map(a => Object.assign(a, {
                incorrect: zuoraTotals.filter(t => t[0] === a['Account Number'])[0] || [],
                subscription: JSON.stringify(recordsToValidate.filter(r => r['Account.AccountNumber'] === a['Account Number']).map(s => s['ProductRatePlan.Name'])),
//                    multiplePrimary:  TODO: check if selecting one product fixes the price
            }))
            const incorrectMultipleSubs = displayIncorrect.filter(a => {
                const subs = recordsToValidate.filter(r => r['Account.AccountNumber'] === a['Account Number']);
                const productGroups = _.groupBy(subs, e => e['ProductRatePlan.Name']);
                return Object.keys(productGroups).length === 0 || Object.keys(productGroups).filter(k => productGroups[k].length > 1).length > 0;
            })
            const verifiableTotal = (correct.length + incorrect.length) - incorrectMultipleSubs.length;
            console.log(incorrectMultipleSubs.length + ' incorrect due to multiple subscriptions, correct minus multiple: '
                        + correct.length + ' out of ' + verifiableTotal + ' - ' + Math.round(correct.length / verifiableTotal * 100) + '%');
            const unaccounted = displayIncorrect.filter(a => !incorrectMultipleSubs.includes(a));
            console.log(unaccounted.length + ' unaccounted for');
            console.log('incorrect sample (' + displayIncorrect.length + '): ');
            console.log(unaccounted); //.map(r => r.subscription)
            
            return zuoraTotals;
        });
}

if(typeof $$ !== 'undefined') {
    $$.async();
    // TODO: pull zuora product catalog
    compareRecordsCatalog()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
    // TODO: this takes to long to download, describe blocks?
}



reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[11]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[2]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[0]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[4]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[14]
4749 ROR accounts with renewing subscr

[ [ 'A00072949', 264, undefined, 1 ],
  [ 'A00073718', 420, undefined, 1 ],
  [ 'A00074995', 264, undefined, 1 ],
  [ 'A00072250', 264, undefined, 1 ],
  [ 'A00072376', 840, undefined, 2 ],
  [ 'A00072671', 264, undefined, 1 ],
  [ 'A00072830', 2520, undefined, 6 ],
  [ 'A00072413', 264, undefined, 1 ],
  [ 'A00072263', 209, undefined, 1 ],
  [ 'A00072849', 1260, undefined, 3 ],
  [ 'A00079795', 4620, undefined, 11 ],
  [ 'A00072427', 2376, undefined, 9 ],
  [ 'A00075509', 209, undefined, 1 ],
  [ 'A00076391', 696, undefined, 4 ],
  [ 'A00080019', 420, undefined, 1 ],
  [ 'A00078280', 420, undefined, 1 ],
  [ 'A00076768', 264, undefined, 1 ],
  [ 'A00072432', 5280, undefined, 20 ],
  [ 'A00074945', 420, undefined, 1 ],
  [ 'A00072895', 264, undefined, 1 ],
  [ 'A00078019', 946, undefined, 4 ],
  [ 'A00078500', 1320, undefined, 5 ],
  [ 'A00073672', 836, undefined, 4 ],
  [ 'A00075877', 2112, undefined, 8 ],
  [ 'A00072781', 836, undefined, 4 ],
  [ 'A00075301', 528, undefined, 2 ],
  [


readme?

# Install

http://www.oracle.com/technetwork/java/javase/downloads/index.html

https://jenkins.io/doc/pipeline/tour/getting-started/

https://nodejs.org

http://docs.aws.amazon.com/cli/latest/userguide/installing.html


Or use Docker

https://github.com/jenkinsci/docker

`docker run -p 8080:8080 -p 50000:50000 -n jenkins jenkins/jenkins:lts`


`docker exec -it -u root jenkins wget -O - https://deb.nodesource.com/setup_8.x | bash && apt-get install -y nodejs zip aws`


# Setup

Create a pipeline and copy the Jenkinsfile, or import from Github

After 61 build tries you should have a screen that looks all green!


# Test

`npm run test`

# Deploy

Create an access key for AWS: https://console.aws.amazon.com/iam/home?nc2=h_m_sc#/security_credential

http://docs.aws.amazon.com/AWSGettingStartedContinuousDeliveryPipeline/latest/GettingStarted/CICD_Jenkins_Pipeline.html

`aws configure`

`aws lambda update-function-code --function-name eloqua_test --zip-file fileb://index.zip`



Get account information

In [3]:
var _ = require('lodash');
var request = importer.import('request polyfill');
var fs = require('fs');
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString().trim());

function getAuthHeaders(zuoraConfig) {
    return {
        'Content-Type': 'application/json',
        'apiAccessKeyId': zuoraConfig.rest_api_user,
        'apiSecretAccessKey': zuoraConfig.rest_api_password,
        'Accept': 'application/json'
    };
}

if(typeof $$ !== 'undefined') {
$$.async();
request({
    followAllRedirects: true,
    uri: zuoraConfig.rest_api_url + '/action/query',
    json: {
        queryString: 'SELECT Id, Status, Name, Currency FROM Account WHERE AccountNumber=\'A00072263\''
    },
    method: 'POST',
    headers: getAuthHeaders(zuoraConfig)
})
    .then(r => {
        console.log(r.body.records);
        return request({
            followAllRedirects: true,
            uri: zuoraConfig.rest_api_url + '/action/query',
            json: {
                queryString: 'SELECT Id, Status, TermEndDate FROM Subscription WHERE AccountId=\'' + r.body.records[0].Id + '\''
            },
            method: 'POST',
            headers: getAuthHeaders(zuoraConfig)
        })
    })
    .then(r => {
        console.log(r.body.records);
        return request({
            followAllRedirects: true,
            uri: zuoraConfig.rest_api_url + '/action/query',
            json: {
                queryString: 'SELECT Id, Name, SubscriptionId FROM RatePlan WHERE SubscriptionId=\'' + r.body.records.map(r => r.Id).join('\', OR SubscriptionId=\'') + '\''
            },
            method: 'POST',
            headers: getAuthHeaders(zuoraConfig)
        })
    })
    .then(r => _.groupBy(r.body.records, r => r.SubscriptionId))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
}


[ { Currency: 'USD',
    Status: 'Active',
    Name: 'Kathryn Hurley',
    Id: '2c92a0fe5316fd1d01532db19d074c78' } ]
[ { Status: 'Expired', Id: '2c92a0fe5316fd1d01532db19d344c7a' },
  { Status: 'Active',
    TermEndDate: '2018-03-01',
    Id: '2c92a0fe5af9a6b8015b12f22a6b3b37' } ]


{ '2c92a0fe5316fd1d01532db19d344c7a': 
   [ { SubscriptionId: '2c92a0fe5316fd1d01532db19d344c7a',
       Name: 'Act! Premium - Annual Subscription (Loyalty Pricing)',
       Id: '2c92a0fe5316fd1d01532db19d534c7e' } ],
  '2c92a0fe5af9a6b8015b12f22a6b3b37': 
   [ { SubscriptionId: '2c92a0fe5af9a6b8015b12f22a6b3b37',
       Name: 'Act! Premium - Annual Subscription (Loyalty Pricing)',
       Id: '2c92a0fe5af9a6b8015b12f22a833b3b' } ] }